In [3]:

import time
import requests
from bs4 import BeautifulSoup
import pymongo
import pandas as pd
import numpy as np
import re

from sklearn.externals import joblib

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnlzer = WordNetLemmatizer()


from pprint import pprint
from time import time
import logging


import sys
import os
import csv
import shutil
from collections import defaultdict

from sklearn.linear_model import SGDClassifier, Perceptron
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score
#import parfit.parfit as pf


import matplotlib.pyplot as plt

source_path = 'test_data'
#source_file = 'test_strains_reviews.csv'
target_path = '/Users/jordanweil/green_rex/test_data'

In [4]:
mc = pymongo.MongoClient()  # Connect to the MongoDB server using default settings
db = mc['strain_reviews']  # Use (or create) a database called 'election_predictions'
docs = db['review']  # Use (or create) a collection called 'docs'

In [5]:
docs.count_documents(filter={})

0

In [6]:
docs.drop()

## Here is how i want the scrape to work

    """LOS = [List of Strains]
        for s in LOS:
        url = "https://www.leafly.com/{}/reviews?page=".format(s)
        for i in range(1,100):
            d = get_reviews(url+str(i))
            st, rev = parse_docs(d)
            if len(st) > 0:
               append.(add to your dictionary)
            else:
                break"""

In [7]:
def table_check2(l):
    """This checks to see if a url is already in my database of reviews. 
    Only needs to be run in the get reviews function"""
    for url in l:
        if docs.find_one({'url':url}):
            None
        else:
            docs.insert_one({'url': url,
                 'html': requests.get(url).content
                 })

In [8]:
def get_reviews2(l):
    """Pass in a list of URL's and return them in a mongo db table as a dicitonary with 
    {'url', 'html'} and their corresponding values"""
    r = requests.get(l)
    html = (r.content)
    return html

In [9]:
def get_stars_list(d):
    stars = []
    for key, values in d.items():
        soup = BeautifulSoup(values, 'html.parser')
        tags = soup.select("div.div.stars")
        for t in tags:
            stars.append(t.attrs['style'])
    return stars

In [10]:
def star_int_conv(l):
    flat_stars= []
    star_num = []
    for s in l:
        star = int((s[6:].split(';')[0]).strip('px'))
        star_num.append((star/22))
    return star_num

In [11]:
def list_o_strains(i):
    LOS = []
    r = requests.get(i)
    soup2 = BeautifulSoup(r.content, 'html.parser')
    strains = soup2.find_all('a', class_="ga_Explore_Strain_Tile")
    for s in strains:
         LOS.append(str(s.attrs['href'])[1:])
    return LOS

In [12]:
def parse_docs2(d):
    """Parse the HTML docs that we have stored in a dictionary, return as a list.
    Also scrape and parse star rating for each review """
    strain_text= []
    star_rate = []
    user_name = []
    soup = BeautifulSoup(d, 'html.parser')
    revs = soup.find_all('p',class_='strain-review__text') 
    for r in revs:
        text = r.text
        remove_punch = re.sub('[^A-Za-z ]' , "" ,text )
        token = remove_punch.lower().split()
        srm_token = [wnlzer.lemmatize(i) for i in token if not i in set(stopwords.words('english'))]
        clean_text = " ".join(srm_token)
        strain_text.append(clean_text)
    tags = soup.select("div.div.stars")
    star = []
    for t in tags:
        star.append(t.attrs['style'])
    star_rate.append(star_int_conv(star))
    users = soup.find_all('div', class_='strain-review__title')
    for u in users:
        temp = u.find('h2')
        user_name.append(temp.text)
    return star_rate, strain_text, user_name

In [13]:
"""#Test works better, is updated!
def scraper_dummy1(los):
    los_urls = []
    rev_urls = []
    for s in los:
        url = "https://www.leafly.com{}/reviews?page=".format(s)
        print(url)
        for i in range(1,100):
            rev_urls=url+str(i)
            d = get_reviews2(rev_urls)
            star, reviews, users = parse_docs2(d)
            print(users)
            if len(star[0]) == 0:
                break"""

'#Test works better, is updated!\ndef scraper_dummy1(los):\n    los_urls = []\n    rev_urls = []\n    for s in los:\n        url = "https://www.leafly.com{}/reviews?page=".format(s)\n        print(url)\n        for i in range(1,100):\n            rev_urls=url+str(i)\n            d = get_reviews2(rev_urls)\n            star, reviews, users = parse_docs2(d)\n            print(users)\n            if len(star[0]) == 0:\n                break'

In [14]:
def scraper_dummy_test(los):
    los_urls = []
    rev_urls = []
    user_list_of_lists = []
    star_list_of_lists = []
    reviews_list_o_lists = []
    #grab URL landing pages for reviews on each strain.
    for s in los:
        url = "https://www.leafly.com/{}/reviews?page=".format(s)
        strain_star_list = []
        strain_user_list = []
        strain_rev_list = []
        #get all reviews, stars, and users for each strain in list of strains
        for i in range(1,100):
            rev_urls=url+str(i)
            d = get_reviews2(rev_urls)
            star, reviews, users = parse_docs2(d)
            #return stars, strains, reviews, in one list, not list of lists for each page scraped. 
            #star_unlisted = 
            strain_star_list.extend([x for x in star[0]])
            strain_user_list.extend(users)
            strain_rev_list.extend(reviews)
            if len(star[0]) == 0:
                break
        #these are long single lists, not list of lists
        star_list_of_lists.append(strain_star_list)
        user_list_of_lists.append(strain_user_list)
        reviews_list_o_lists.append(strain_rev_list)
        #make into dictionaries 
        for s in los:
            star_ds = []
            rev_ds = []
            for i in range(len(star_list_of_lists)):
                user_revs =  dict(list(zip(user_list_of_lists[i], reviews_list_o_lists[i])))
                rev_ds.append(user_revs)
                user_stars = dict(list(zip(user_list_of_lists[i], star_list_of_lists[i])))
                star_ds.append(user_stars)
    #create Pandas DF of each dictionary as well.
    star_dfs = []
    rev_dfs = []
    for s in star_ds:
        star_dfs.append(pd.DataFrame.from_dict(s, orient='index')) 
    for r in rev_ds: 
        rev_dfs.append(pd.DataFrame.from_dict(r, orient ='index'))
    #create large dataframe
    star_df = pd.concat(star_dfs, axis=1, sort=True)
    rev_df = pd.concat(rev_dfs, axis=1, sort=True)
    rev_df.columns = los
    star_df.columns = los
    return star_df, rev_df

In [15]:
test_strains = list_o_strains('https://www.leafly.com/explore')

In [16]:
test_strains

['hybrid/blue-dream',
 'sativa/sour-diesel',
 'hybrid/gsc',
 'sativa/green-crack',
 'hybrid/og-kush',
 'indica/granddaddy-purple',
 'hybrid/original-glue',
 'hybrid/white-widow',
 'hybrid/blue-dream',
 'sativa/sour-diesel',
 'hybrid/gsc',
 'sativa/green-crack',
 'hybrid/og-kush',
 'indica/granddaddy-purple',
 'hybrid/original-glue',
 'hybrid/white-widow',
 'sativa/jack-herer',
 'indica/bubba-kush',
 'hybrid/pineapple-express',
 'hybrid/trainwreck',
 'hybrid/ak-47',
 'sativa/durban-poison',
 'indica/northern-lights',
 'hybrid/headband',
 'indica/blue-cheese',
 'sativa/strawberry-cough',
 'hybrid/chemdawg',
 'indica/purple-kush',
 'sativa/lemon-haze',
 'sativa/super-lemon-haze',
 'indica/grape-ape',
 'indica/blueberry',
 'sativa/alaskan-thunder-fuck',
 'sativa/super-silver-haze',
 'indica/blackberry-kush',
 'hybrid/cherry-pie',
 'indica/master-kush',
 'hybrid/skywalker-og',
 'hybrid/cheese',
 'indica/death-star',
 'sativa/chocolope',
 'sativa/amnesia-haze',
 'hybrid/tahoe-og',
 'sativa/m

In [2]:
stars, reviews = scraper_dummy_test(test_strains)

NameError: name 'scraper_dummy_test' is not defined

In [1]:
stars

NameError: name 'stars' is not defined

In [61]:
stars.to_csv("/Users/jordanweil/green_rex/new_full_test_strains_stars.csv")

In [62]:
reviews.to_csv("/Users/jordanweil/green_rex/new_full_test_strains_reviews.csv")

In [63]:
test_stars = pd.read_csv("new_full_test_strains_stars.csv")

In [64]:
test_reviews = pd.read_csv("new_full_test_strains_reviews.csv")

In [68]:
tr = test_reviews.drop('Unnamed: 0', axis=1)

In [69]:
tr

,hybrid/blue-dream,sativa/sour-diesel,hybrid/gsc,sativa/green-crack,hybrid/og-kush,indica/granddaddy-purple,hybrid/original-glue,hybrid/white-widow,hybrid/blue-dream.1,sativa/sour-diesel.1,...,indica/gods-gift,indica/la-confidential,hybrid/agent-orange,indica/purple-urkle,hybrid/lemon-kush,hybrid/mazar-x-blueberry,hybrid/golden-goat,indica/afghan-kush,hybrid/dutch-treat,indica/hindu-kush
0,NaN,NaN,NaN,one little intense normally stick high cbd str...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,loved la confidential tookall stress frommy back,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,new cannabis user high twice friend trying str...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,purchased sqdc quebec blue dream aurora thc bo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,purchased sqdc quebec blue dream aurora thc bo...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,best looking bud opinion im love dense bud lik...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,best looking bud opinion im love dense bud lik...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ehlers danlos syndrome constantly battling chr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ehlers danlos syndrome constantly battling chr...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,made paranoid felt like dying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,tried first time today took two small hit vape...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tried first time today took two small hit vape...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
